<a href="https://colab.research.google.com/github/stanbuklovskyi/algorithms-from-scratch/blob/main/FromScratch_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is an OOP style code for Random Forest ML algorithm
This code uses FromScratch_DecisionTree (my own implementation of
DecisionTree class that can be found in the same repository)

To use in this code:
- Download "FromScratch_DecisionTree.py" file to your google drive 
- Move to you working directory so it can be imported 

In [1]:
# acess the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# move .py file with our classes
!cp /content/drive/MyDrive/Colab\ Notebooks/FromScratch_DecisionTree.py /content

In [3]:
# import needed libraries and classes from the file
from FromScratch_DecisionTree import DecisionTree
import numpy as np
from collections import Counter

In [ ]:
# def bootstrap function (feeding random subset of the data)
def bootstrap_sample(X,y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, size=n_samples, replace=True)

    return X[idxs], y[idxs]

# def most_common_label function
def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]

    return most_common

In [ ]:
# random forest code
class RandomForest:
    
    def __init__(self, n_trees=100, min_samples_split=2, max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = [] # store trees here

    def fit(self, X, y):
        self.tress = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                min_samples_split = self.min_samples_split,
                max_depth = self.max_depth,
                n_features = self.n_feats
            )

            # pass random subset of data to a tree
            X_sample, y_sample = bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        # to get corresponding predictions of earch tree we need to...
        # ... swap axis in the array
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]

        return np.array(y_pred)



In [ ]:
# testing the class
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# load breast cancer dataset
data = datasets.load_breast_cancer()
X, y = data.data, data.target

# train, test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

# get our decision tree
clf = RandomForest(n_trees = 3)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, predictions)
print(acc)

0.9298245614035088
